# FINM 32000 Numerical Methods HW1

In [27]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import bisect, brentq
from copy import copy

## Part I: Contract 

### 1. Vanilla Option
- arrtribute: $K$, $T$, $c$ (option price, default to be None -- unknown)

In [28]:
class CallOption:

    def __init__(self, K, T, price=None):
        self.K = K
        self.T = T
        self.price = price

In [29]:
class PutOption:

    def __init__(self, K, T, price=None):
        self.K = K
        self.T = T
        self.price = price

### 2. Barrier Option
- arrtribute: $K$, $T$, $H$, $\mathcal{T}$ (given as observation interval, $\Delta \mathcal{T}$)

1. An up-and-out put on S with strike K, expiry T, barrier H and ovservation times $\mathcal{T}$ pays:
$$ (K-S_T)^{+} \mathbb{1}_{max_{t\in\mathcal{T}}S_t<H}$$

In [30]:
class UpAndOutPut:
    
    def __init__(self, K, T, barrier, observationinterval):
        self.K = K
        self.T = T
        self.barrier = barrier
        self.observationinterval = observationinterval

## Part II: Diffusion Dynamics

 ```GBMdynamics```
- attribute: $S_0$, $r$, $R_{grow}$, $\sigma$
- method: ```update_sigma```, to change the sigma attribute, used in the later ```AnalyticEngine``` to find implied volatility

In [31]:
class GBMdynamics:

    def __init__(self, S, r, rGrow, sigma=None):
        self.S = S # S0
        self.r = r # interest rate
        self.rGrow = rGrow # R_grow
        self.sigma = sigma # instantenous vol

    def update_sigma(self, sigma):
        self.sigma = sigma
        return self

## Part III: Engine

### 1. ```TreeEngine```

- #### Diffusion Dynamics
Diffusion is: (where $W$ is $\mathbb{P}$-$BM$)
$$ dS_t = R_{grow} S_t dt + \sigma S_t dW_t$$
So $X:=logS$ has dynamics:
$$ dX_t = \upsilon dt + \sigma dW_t$$ 
where $$ \upsilon := R_{grow}-\frac{\sigma^2}{2} $$

- #### Trinomial Tree Approximation
$$ p_{u,d} = \frac{1}{2} [ \frac{\sigma^2 \Delta t + \upsilon^2 (\Delta t)^2}{(\Delta x)^2} \pm \frac{\upsilon \Delta t}{\Delta x}]$$
$$ p_m = 1 - \frac{\sigma^2 \Delta t + \upsilon^2 (\Delta t)^2}{(\Delta x)^2}$$

- #### Grid
1.Decide $\Delta t$:

$$\Delta t = \frac{T}{N}$$
thus time grid:
$$ \Rightarrow t_n = n \Delta t \quad n = 0,1,...,N$$

- Choose $N$ based on what accuracy you want, note that you want to put obervation times into the nodes, i.e.:
 $$ \frac{\text{observation interval}}{\Delta t} = \frac{\text{observation interval}\times N}{T} = \text{int} $$

2.Decide $\Delta x$: 
$$ logS_0 + j \Delta x = x_j = logS_j \quad j = -N,...,N$$

thus stock price grid:
$$ \Rightarrow S_j = S_0 e^{j \Delta x}$$
- Rule of Gold Thumb: $\Delta x = \sigma \sqrt{3 \Delta t}$, we can have a rough $\Delta x$ through this rule, however, we want to make sure that we put the barrier into the halfway of some two consecutive levels (j and j+1). Thus, we firstly calculate j using the rough $\Delta x$, and let it be an integer. Then we use this j to calculate the final $\Delta x$.
$$
\begin{cases}
log S_0 +(j+0.5)\Delta x = log H \\
\Delta x = \sigma \sqrt{3 \Delta t}
\end{cases}
\Rightarrow 
\begin{cases}
j =  \lceil \frac{log(H/S_0)}{\sigma \sqrt{3 \Delta t}} - \frac{1}{2} \rceil \\
\Delta x = \frac{log(H/S_0)}{j+\frac{1}{2}}
\end{cases}


In [32]:
class TreeEngine:

    def __init__(self, N):
        self.N = N

    def price_upandout(self, dynamics, contract):

        deltat = contract.T / self.N
        J = np.ceil(np.log(contract.barrier/dynamics.S)/(dynamics.sigma*np.sqrt(3*deltat))-0.5)
        deltax = np.log(contract.barrier/dynamics.S)/(J+0.5)

        Sgrid = dynamics.S*np.exp(np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True)*deltax)
        # np.linspace(self.N, -self.N, num=2*self.N+1, endpoint=True) generate 2N+1 points from N, N-1, ... to -N
        # Here I decided to make the SMALLER indexes in this array correspond to HIGHER S

        numTimestepsPerObs = contract.observationinterval/deltat
        if abs(numTimestepsPerObs-round(numTimestepsPerObs)) > 1e-8:
            # if numTimestepsPerObs is not int
            raise ValueError("This value of N fails to place the observation dates in the tree.")

        nu = dynamics.rGrow - 0.5 * (dynamics.sigma**2)  
        Pu = 0.5 * (((dynamics.sigma**2) * deltat + (nu**2)*(deltat**2))/(deltax**2) + nu*deltat/deltax)     
        Pd = 0.5 * (((dynamics.sigma**2) * deltat + (nu**2)*(deltat**2))/(deltax**2) - nu*deltat/deltax)       
        Pm = 1 - ((dynamics.sigma**2) * deltat + (nu**2)*(deltat**2))/(deltax**2)     

        # terminal case: suppose T is not in observation times
        optionprice = np.maximum(contract.K-Sgrid,0)   # an array of time-T option prices.

        # Next, induct backwards to time 0, updating the optionprice array
        # Hint: if x is an array, then what are x[2:] and x[1:-1] and x[:-2]
        for t in np.linspace(self.N-1, 0, num=self.N, endpoint=True)*deltat: # from (N-1)*deltat to 0
            Sgrid = Sgrid[1:-1] # shrinking Sgrid (the most up and the most down node are deleted) (-2)
            optionprice = np.exp(-dynamics.r*deltat)*(Pu*optionprice[:-2] + Pm*optionprice[1:-1] + Pd*optionprice[2:]) # shrinking (-2)
            if t in np.linspace(self.N-1, 0, num=self.N, endpoint=True)*deltat*numTimestepsPerObs: # if t is an observation time
                for j in range(0, len(Sgrid)):
                    if Sgrid[j] >= contract.barrier:
                        optionprice[j] = 0
            
        return optionprice[0]
        # The [0] is assuming that we are shrinking the optionprice array in each iteration of the loop,
        # until finally there is only 1 element in the array.
        # If instead you are keeping unchanged the size of the optionprice array in each iteration,
        # then you need to change the [0] to a different index.


### 2. ```AnalyticEngine```

- All methods should be provided with 2 arguments: ```dynamics```, ```contract```
- ```BSpriceCall```: calculate the European Call price $c$ throgh BSM given $S, T, K, R_{grow}, \sigma$
    - BSM for European call:
    $$ c(S_t, t) = S_tN(d_1) - Ke^{-r(T-t)}N(d_2) =  e^{-r(T-t)}[F_tN(d_1)-KN(d_2)]$$
    $$ F_t = S_te^{r(T-t)}$$
    $$ d_{1,2} = \frac{log(S_t/K) + (r\pm\frac{1}{2}\sigma^2)(T-t)}{\sigma\sqrt{T-t}} $$
    
- ```BSpricePut```: calculate the European Put price $p$ throgh BSM given $S, T, K, R_{grow}, \sigma$
    - use put-call parity:
    $$ c(S_t, t) - p(S_t, t) = e^{-r(T-t)}(F_t - K) = S_t - Ke^{-R(T-t)}$$
    - or BSM for European put:
    $$ p(S_t, t) = Ke^{-r(T-t)}N(-d_2) - S_tN(-d_1) = e^{-r(T-t)}[KN(-d_2)-F_tN(d_1)]$$
    - note that the first equation holds forever, while the second equation holds without dividend


- ```IV```: solve the $\sigma_{imp}$ given the Call market price $c$ and 
    - non-arbitrage condition:
    $$ [e^{-r(T-t)}(F-K)]^+ = [S_t - Ke^{-r(T-t)}]^+ < c(S_t, t) < S_t $$
    - try logic:
    $$ \sigma \uparrow \quad \Rightarrow \quad c \uparrow$$
    1. firstly find the lower and upper bound of $\sigma_{imp}$ by trying an intital value 0.2 and plug it into the BSM formula (```BSpriceCall```) (note that the class ```dynamics``` has a method of ```update_sigma```), then we use 2 ```while``` cycles:
    
        - if the derived call price is higher than actual price, then we enter into the 1st while cycle. Since a higher derived price means that we have guessed a bigger $\sigma_{imp}$, thus we divide it by 2 until the derived price falls below the actual price. Then we get a lower bound of $\sigma_{imp}$ and enter into the 2nd while cycle. The 2nd while cycle then multiply this lower bound by 2 (note that 1 time is enough) and get the upper bound of $\sigma_{imp}$ and exit the cycle.
        
        - if the derived call price is lower than actual price, then we directly enter into the 2nd while cycle. Since a lower price means that we have guessed a samller $\sigma_{imp}$, thus we multiple it by 2 until the derived price falls above the actual price. Then we get the upper bound of $\sigma_{imp}$ and exit the cycle.
    2. After we get a rough range of $\sigma_{imp} \in [\text{lo, hi}]$, we then use bisection algorithm to get an accurate $\sigma_{imp}$
        

In [42]:
class AnalyticEngine:

    def __init__(self):
        pass

    def BSpriceCall(self, dynamics, contract):
        # ignores contract.price if given, because this function calculates price based on the dynamics

        F = dynamics.S*np.exp(dynamics.rGrow*contract.T)
        std = dynamics.sigma*np.sqrt(contract.T)
        d1 = np.log(F/contract.K)/std+std/2
        d2 = d1-std
        call_price = np.exp(-dynamics.r*contract.T)*(F*norm.cdf(d1)-contract.K*norm.cdf(d2))
        return call_price
    
    def BSpricePut(self, dynamics, contract):
        F = dynamics.S*np.exp(dynamics.rGrow*contract.T)
        std = dynamics.sigma*np.sqrt(contract.T)
        d1 = np.log(F/contract.K)/std+std/2
        d2 = d1-std
        # call_price =  np.exp(-dynamics.r*contract.T)*(F*norm.cdf(d1)-contract.K*norm.cdf(d2))
        # put_price = call_price - np.exp(-dynamics.r*contract.T)*(F-contract.K)
        put_price = np.exp(-dynamics.r*contract.T)*(contract.K*norm.cdf(-d2)-F*norm.cdf(-d1))
        return put_price

    def IV(self, dynamics, contract):
        # ignores dynamics.sigma, because this function solves for sigma.

        if contract.price is None:
            raise ValueError('Contract price must be given')

        # non-arbitrage condition
        df = np.exp(-dynamics.r*contract.T)  # discount factor
        F = dynamics.S / df
        lowerbound = np.max([0,(F-contract.K)*df])
        C = contract.price
        if C<lowerbound:
            return np.nan
        if C==lowerbound: # if C=upperbound=S=F*df, then implied vol is infinity
            return 0
        if C>=F*df:
            return np.nan

        dytry = copy(dynamics)
        # We "try" values of sigma until we find sigma that generates price C

        # First find lower and upper bounds
        sigma_try = 0.2
        while self.BSpriceCall(dytry.update_sigma(sigma_try),contract)>C:
            sigma_try /= 2
        while self.BSpriceCall(dytry.update_sigma(sigma_try),contract)<C:
            sigma_try *= 2
        hi = sigma_try
        lo = hi/2
        # We have calculated "lo" and "hi" which bound the implied volatility from below and above.
        # In other words, the implied volatility is somewhere in the interval [lo,hi].
        # Then, to calculate the implied volatility within that interval,
        # for purposes of this homework, you may either (A) write your own bisection algorithm,
        # or (B) use scipy.optimize.bisect or (C) use scipy.optimize.brentq
        # You will need to provide lo and hi to those solvers.
        # There are other solvers that do not require you to bound the solution
        # from below and above (for instance, scipy.optimize.fsolve is a useful solver).
        # However, if you are able to bound the solution (of a single-variable problem),
        # then bisection or Brent will be more reliable.

        price_diff = lambda sigma: self.BSpriceCall(dytry.update_sigma(sigma), contract) - C # the function that plug into the bisection algorithm

        # (A) write your own bisection algorithm
        def my_bisection_method(f, left, right, tol=1e-5, max_iter=100000):
            if f(left) * f(right) >= 0:
                raise ValueError("f(a) and f(b) must have opposite signs")
            
            iter_count = 0
            while (right - left) / 2 > tol:
                iter_count += 1

                mid = (left + right) / 2
                if f(mid) == 0 or (right - left) / 2 < tol:
                    # print(f"Root found after {iter_count} iterations: {c}")
                    return mid
                elif f(left) * f(mid) < 0:
                    right = mid
                else:
                    left = mid

                if iter_count >= max_iter:
                    # print(f"Maximum iterations reached. Approximate root: {c}")
                    return mid
                
            # print(f"Root found after {iter_count} iterations: {(left + right) / 2}")
            return (left + right) / 2

        impliedVolatility = my_bisection_method(price_diff, lo, hi)

        # (B) use scipy.optimize.bisect
        # impliedVolatility = bisect(price_diff, lo, hi)
        
        # (C) use scipy.optimize.brentq
        # impliedVolatility = brentq(price_diff, lo, hi)
        
        return impliedVolatility


##### Example:

In [ ]:
# Test the BSpriceCall function
hw1analytic = AnalyticEngine()
dynamics2 = GBMdynamics(sigma=0.4, rGrow=0, S=100, r=0)
contract2 = CallOption(K=100, T=0.5)
hw1analytic.BSpriceCall(dynamics2,contract2)

11.246291601828489

In [ ]:
# Test the IV function
contract2.price = 12
hw1analytic.IV(dynamics2,contract2)    # This code, EXACTLY AS WRITTEN HERE, must execute without crashing

0.427008056640625

## Part IV: Problems and Answers

### Problem 1 and Answers
#### 1.(a)

In [45]:
hw1contract = UpAndOutPut(K=95, T=0.25, barrier=114, observationinterval=0.02)
# in this example, observation times Tau = {0.02, 0.04, ..., 0.24}

hw1dynamics = GBMdynamics(S=100, sigma=0.4, rGrow=0, r=0)

In [46]:
hw1tree=TreeEngine(N=20000)

up_and_out_price = hw1tree.price_upandout(hw1dynamics, hw1contract)
print(f"Up-and-out put option price is {round(up_and_out_price,4)}")

Up-and-out option price is 5.3087


#### 1.(b)


In [49]:
put_contract = PutOption(K=95, T=0.25)

hw1dynamics = GBMdynamics(S=100, sigma=0.4, rGrow=0, r=0)

In [50]:
hw1analytic = AnalyticEngine()

put_price = hw1analytic.BSpricePut(hw1dynamics,put_contract)

up_and_in_price = put_price - up_and_out_price
print(f"Up-and-in option put price is {round(up_and_in_price,4)}")

Up-and-in option price is 0.2109


##### <span style = "background-color: blue"> Barrier Option: up-and-in = vanilla - up-and-out</span> 

#### 1.(c)

(c1) the time-0 price of the continuously monitored barrier option is smaller than than the time-0 price of the discretely monitored option. With continuously monitoring, the option has a bigger probability of being knocked out (maximum reaching H), thus has a bigger probability ending up with payoff=0.

(c2) 
- $\alpha = \frac{95}{114} = \frac{5}{6}$
- the time-0 price of the continuously monitored barrier option is $5.0315$

![Problem1 (c2)](https://github.com/Sally-Yitong/FINM-32000-Numerical-Methods/blob/main/HW1/FINM%2032000%20HW1%20Problem1%20c2.jpg?raw=true)

PS: 
- under $r=0$, given continuous up-and-out put $H=114$, $K=95$, replicate by:
    - long 1 share of vanilla put $K_B=95$ 
    - short $\frac{K}{H}=\frac{95}{114}$ share of vanilla call $K_A=\frac{H^2}{K_B}=\frac{114^2}{195}=136.8$
- another solution -- When the stock price S hits the barrier price, the intrinsic value of the puts in the portfolio (strike price 95) will be equal and opposite to the intrinsic value of alpha units of call options (strike price 136.8)

$$\alpha = \frac{K_B - H}{H - K_A} = \frac{H}{K_B}$$

In [57]:
call_A = CallOption(K=136.8, T=0.25)
put_B = PutOption(K=95, T=0.25)

hw1dynamics = GBMdynamics(S=100, sigma=0.4, rGrow=0, r=0)

In [56]:
hw1analytic = AnalyticEngine()

call_A_price = hw1analytic.BSpriceCall(hw1dynamics,call_A)
put_B_price = hw1analytic.BSpricePut(hw1dynamics,put_B)

alpha = 95/114

continuous_up_and_out_price = put_B_price - alpha*call_A_price
print(f"Continuous up-and-out put option price is {round(continuous_up_and_out_price,4)}")

Continuous up-and-out put option price is 5.0315


##### <span style = "background-color: blue"> Replication of a continuosly monitoring barrier option: </span>

Using a portfolio of vanilla puts and calls, key point is when the stock hit the barrier, the value of the puts and calls cancel each other 
Assumption: $\textcolor{#FF0000}{r = 0}$ (since we don't know when will the stock hit the barrier)

The continuously monitored barrier option which pays $(95-S_T)^+ \mathbf{1}(max_{0 \leq t \leq T} S_t < 114)$ can be replicated by a portfolio of T-expiry options: long 1 plain vanilla put struck at 95, and short plain $\alpha = \frac{95}{114} = \frac{5}{6}$ vanilla calls struck at 136.8

- If S does not hit the barrier before time T, then simply collect the time-T payout of the 95 put, as desired. 

- If S does hit the barrier, then at the time when S is at the barrier, the 1 unit of the vanilla put has value that exactly cancels the value of the $\alpha$ units of the plain vanilla call; so at that time, we close out the portfolio positions,for a net payment of zero, as desired.

### Problem 2 and answers
#### 2.(a) 
Interest rate is $0$. A non-dividend-paying stock S has time-0 price $S_0 = 100$. At time 0, you
observe the dollar prices of at-the-money ($K = 100$) European calls on S at $0.5$-year and
$1$-year expiries to be $11.25$ and $12.00$, respectively. Find  the  time-0  Black-Scholes  implied  volatilities  of  these  two  options.

In [33]:
analytic = AnalyticEngine()
contract1 = CallOption(K=100, T=0.5, price=11.25)
contract2 = CallOption(K=100, T=1, price=12.00)
dynamics = GBMdynamics(S=100, r=0, rGrow=0)
sigma1 = analytic.IV(dynamics,contract1) 
sigma2 = analytic.IV(dynamics,contract2) 
print(f'implied vol for option1 is {round(sigma1,4)} and option2 is {round(sigma2,4)}.')

implied vol for option1 is 0.4001 and option2 is 0.3019.


#### 2.(b) 
Consider an at-the-money European call on S with expiry $0.75$.  Suppose that you try to price it by assuming that its implied volatility is equal to the midpoint (the arithmetic average) ofthe $0.5$-expiry and the $1.0$-expiry implied volatilities. Under that assumption, what would be the time-0 price of the $0.75$-expiry call?

In [35]:
analytic = AnalyticEngine()
sigma3 = 0.5 * (sigma1+sigma2)
contract3 = CallOption(K=100, T=0.75)
dynamics = GBMdynamics(sigma = sigma3, S=100, r=0, rGrow=0)
price3 = analytic.BSpriceCall(dynamics,contract3)
print(f'price of option3 is {round(price3,4)}.')

price of option3 is 12.0814.


#### 2.(c)
The price computed in (b) would allow arbitrage involving the $0.75$-expiry call and one of the other contracts. Describe the steps of this arbitrage. You may assume either cash settlement or physical settlement of these options, but specify what your assumption is. 

Cash settlement of an in-the-money option at time T means that you receive $S_T−K$ dollars if you are long the contract,$K−S_T$ dollars if you are short the contract.  Physical settlement of an in-the-money option at time T means that you receive $1$ share of stock and $−K$ dollars if you are long the contract, or $−1$ share of stock and $+K$ dollars if you are short the contract.


Answer:

note that $12.0814 > 12$, which shows that $0.75$-expiry call price > $1$-expiry call price, thus conduct arbitrage: (assume <font color='red'>**physical settlement** </font>)
- short 1 share of $0.75$-expiry call
- long 1 share of $1$-expiry call

    - at $t=0$: 
        - $V_0 = -12.0814 + 12 = -0.0814 <0$

    - at $t=0.75$: (note that we don't have to care about $V_t$ when $t \neq 0$ or $t \neq T$)
        - if $S_{0.75} > K$, then your counterparty execute the $0.75$-expiry call，then you receive $−1$ share of stock and $+K$ dollars
        - if $S_{0.75} <= K$, then your counter party doesn't execute the $0.75$-expiry call and you do nothing

    - at $T=1$:
        - if $S_{0.75} > K$, then $V_1 = (S_1-K)^+ + K - S_1 \geq 0$
        - if $S_{0.75} <= K$, then $V_1 = (S_1-K)^+ \geq 0$

- thus $V_0 <0$ and $P(V_T \geq 0) = 1$, this is a type-II arbitrage

##### <span style = "background-color: blue"> Conclusion: </span> 
Expiry interpolation should <font color='red'>NOT</font> be done by linear interpolation of implied volatility. A better alternative would be linear interpolation of the total implied variance $\textcolor{#FF0000}{\sigma_{imp}^2T}$.